In [1]:
#### SHAP input ######
import os
import numpy as np
import pandas as pd

police_path = 'F:/FacialExpression/Police/TimeSeriesAnalysisResults'
police_output_path = 'F:/FacialExpression/Police/TimeSeriesAnalysisResults/SHAP_input'
stim_path = 'F:/FacialExpression/3rdYear_stim/TimeSeriesAnalysisResults'
stim_output_path = 'F:/FacialExpression/3rdYear_stim/TimeSeriesAnalysisResults/SHAP_input'
mock_path = 'F:/FacialExpression/3rdYear/TimeSeriesAnalysisResults'
mock_output_path = 'F:/FacialExpression/3rdYear/TimeSeriesAnalysisResults/SHAP_input'

ME_list = ['AU01','AU02','AU04','AU05','AU06','AU07','AU09','AU12','AU14','AU15','AU17','AU20','AU23','AU25','AU26']
Anxiety_list = ['AU24','AU28','AU45']
EC_control_list = ['AU01','AU02','AU04','AU05','AU06','AU07','AU09','AU10','AU12','AU14','AU15','AU17','AU20','AU23','AU25','AU26','AU28', 'non_AU45', 'Neutral']

path = mock_path
output_path = mock_output_path

ME_row_indices, ME_col_indices = [], []
Anxiety_row_indices, Anxiety_col_indices = [], []
EC_control_row_indices, EC_control_col_indices = [], []

# Collect row and column indices for p-values <= 0.05
for pval in sorted(os.listdir(path)):
    if pval.split('.')[0].split('_')[-1] == 'pvalue':
        if pval.split('.')[0].split('_')[1] == 'ME':
            df = pd.read_csv(os.path.join(path, pval))
            df = df.to_numpy()
            df = df[:, 1:-1]  # Exclude ME Sum
            ME_row_indices, ME_col_indices = np.where(df <= 0.05)
        elif pval.split('.')[0].split('_')[1] == 'Anxiety':
            df = pd.read_csv(os.path.join(path, pval))
            df = df.to_numpy()
            df = df[:, 1:-1]  # Exclude Blink rate
            Anxiety_row_indices, Anxiety_col_indices = np.where(df <= 0.05)
        elif pval.split('.')[0].split('_')[1] == 'EC':
            df = pd.read_csv(os.path.join(path, pval))
            df = df.to_numpy()
            df = df[:, 2:-1]  # Exclude Gaze
            EC_control_row_indices, EC_control_col_indices = np.where(df <= 0.05)

for f in sorted(os.listdir(os.path.join(path, 'MicroExpression'))):
    result = []
    name = []
    
    # Process MicroExpression data
    df = pd.read_csv(os.path.join(path, 'MicroExpression', f))
    df = df.to_numpy()
    df = df[:, 1:-1]  # Exclude ME Sum
    
    for i in range(np.shape(ME_row_indices)[0]):
        result.append(df[ME_row_indices[i], ME_col_indices[i]])
        frame_num = str(ME_row_indices[i] + 1)
        i_name = 'ME_' + ME_list[ME_col_indices[i]] + '_' + frame_num + 'frame'
        name.append(i_name)
    
    # Process Anxiety data
    df = pd.read_csv(os.path.join(path, 'Anxiety', f))
    df = df.to_numpy()
    df = df[:, 1:-1]  # Exclude Blink rate
    
    for i in range(np.shape(Anxiety_row_indices)[0]):
        result.append(df[Anxiety_row_indices[i], Anxiety_col_indices[i]])
        frame_num = str(Anxiety_row_indices[i] + 1)
        i_name = 'Anxiety_' + Anxiety_list[Anxiety_col_indices[i]] + '_' + frame_num + 'frame'
        name.append(i_name)
    
    # Process EC Control data
    df = pd.read_csv(os.path.join(path, 'EC_control', f))
    df = df.to_numpy()
    df = df[:, 2:-1]  # Exclude Gaze
    
    for i in range(np.shape(EC_control_row_indices)[0]):
        result.append(df[EC_control_row_indices[i], EC_control_col_indices[i]])
        frame_num = str(EC_control_row_indices[i] + 1)
        i_name = 'EC_control_' + EC_control_list[EC_control_col_indices[i]] + '_' + frame_num + 'frame'
        name.append(i_name)
    
    result = np.array(result)
    result_df = pd.DataFrame(result, index=name)
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    result_df.to_csv(os.path.join(output_path, f.split('.')[0] + '.csv'))



KeyboardInterrupt



In [ ]:
#### Calculating SHAP value ####
import os
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_sample_weight
from scipy.sparse import csr_matrix

police_path = 'F:/FacialExpression/Police/TimeSeriesAnalysisResults/SHAP_input'
stim_path = 'F:/FacialExpression/3rdYear_stim/TimeSeriesAnalysisResults/SHAP_input'
mock_path = 'F:/FacialExpression/3rdYear/TimeSeriesAnalysisResults/SHAP_input'
police_output_path = 'F:/FacialExpression/Police/TimeSeriesAnalysisResults/SHAP_Results'
stim_output_path = 'F:/FacialExpression/3rdYear_stim/TimeSeriesAnalysisResults/SHAP_Results'
mock_output_path = 'F:/FacialExpression/3rdYear/TimeSeriesAnalysisResults/SHAP_Results'

path = stim_path
output_path = stim_output_path
DB_type = 'stim'

if DB_type == 'police':
    feat_num = 49
    sub_num = 74
elif DB_type == 'stim':
    feat_num = 25
    sub_num = 72
elif DB_type == 'mock':
    feat_num = 738
    sub_num = 72
    
# Data preprocessing (Setting X and y)
# Here, you should load the data and perform necessary preprocessing to set X and y.
# X: Input data of shape (n_samples, n_features)
# y: Binary classification output data of shape (n_samples,) with labels T or F

X = np.zeros((len(os.listdir(path)), feat_num))
y = np.zeros((len(os.listdir(path))))

feature_names = []
i = 0
for f in sorted(os.listdir(path)):
    df = pd.read_csv(os.path.join(path, f))
    df = df.to_numpy()
    for feat in range(feat_num):
        feature_names.append(df[feat, 0])
    df = df[:,1:]
    label = f.split('.')[0].split('_')[-1]
    if label == 'T':
        label = 0
    elif label == 'F':
        label = 1
    X[i,:] = df[:,0]
    y[i] = label
    i += 1

# Creating Subject labels
subject_labels = np.repeat(range(sub_num), 3)

# Leave-One-Subject-Out cross-validation setup
logo = LeaveOneGroupOut()

# XGBoost model configuration
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss'
}

# List to store SHAP values
shap_values_list = []
accuracies = []  # List to store accuracies
all_true_labels = []  # Store all true labels from all iterations
all_pred_labels = []  # Store all predicted labels from all iterations

# Leave-One-Subject-Out cross-validation
for train_idx, test_idx in logo.split(X, y, groups=subject_labels):
    
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    
    # Creating an XGBoost model and setting class weights
    model = xgb.XGBClassifier(n_estimators=100, max_depth=6,
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
        **params
    )

    # Model training
    model.fit(X_train, y_train)

    # Calculating SHAP values
    explainer = shap.Explainer(model)
    shap_values = explainer.shap_values(X_test)

    shap_values_list.append(shap_values)

    # Model evaluation
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > 0.5).astype(int)
    
    all_true_labels.extend(y_test)
    all_pred_labels.extend(y_pred_binary)
    
    # Calculating accuracy
    accuracy = accuracy_score(y_test, y_pred_binary)
    accuracies.append(accuracy)

# Calculating SHAP values for all subjects
shap_values_ = np.reshape(shap_values_list, (-1, feat_num))

# Calculating mean accuracy
mean_accuracy = np.mean(accuracies)

# Calculating confusion matrix for the entire dataset
conf_matrix = confusion_matrix(all_true_labels, all_pred_labels)

# Calculating accuracy
accuracy = accuracy_score(all_true_labels, all_pred_labels)

# Calculating F1 Score
f1 = f1_score(all_true_labels, all_pred_labels)

# Calculating Precision
precision = precision_score(all_true_labels, all_pred_labels)

# Calculating Recall
recall = recall_score(all_true_labels, all_pred_labels)

# Calculating Sensitivity
sensitivity = conf_matrix[1, 1] / (conf_matrix[1, 1] + conf_matrix[1, 0])

# Calculating Specificity
specificity = conf_matrix[0, 0] / (conf_matrix[0, 0] + conf_matrix[0, 1])

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Sensitivity: {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")

# Generating SHAP summary plot
shap.summary_plot(shap_values_, X, feature_names=feature_names)

# Saving the plot as an image file
if not os.path.exists(output_path):
        os.makedirs(output_path)
plt.savefig(os.path.join(output_path, DB_type + '_SHAP_summary_plot.png'), dpi=300, bbox_inches='tight')
